In [1]:
from numba import cuda, vectorize, float32
from scipy import sparse
import random
import math
import string

import time
import numpy as np

In [2]:
alphabet = string.ascii_lowercase

#alphabet= 'abc'

test_min_str_length_0 = 2
test_max_str_length_0 = 5

number_of_samples_0 = 1000

In [3]:
list_of_strs = [''.join([alphabet[random.randrange(0, len(alphabet))] 
for i in range(random.randrange(test_min_str_length_0, test_max_str_length_0))]) for j in range(number_of_samples_0)]

list_of_chars = set.union(*[set(i) for i in list_of_strs])

In [4]:
char_dict = {value:idx for idx, value in enumerate(sorted(list_of_chars))}

char_dict[' '] = max(char_dict.values())+1

max_length = max([len(i) for i in list_of_strs])

In [5]:
def onehot_string(x, max_length):
    
    if x!=x:
        return None
    
    #Pad to max length
    while len(x)<max_length:
        x+=' '
        
    sparse_matrix = sparse.hstack([sparse.csr_matrix(
        [0 if char_dict[current_char]!=i else 1 for i in range(len(char_dict))]) for current_char in x])
    
    return sparse_matrix


In [6]:
len(list_of_strs)

1000

In [7]:
list_of_strs_0 = list_of_strs[:800]
list_of_strs_1 = list_of_strs[800:]

In [8]:
len(list_of_strs_0), len(list_of_strs_1)

(800, 200)

In [9]:
start_time = time.time()
sparse_strs_0 = sparse.vstack([onehot_string(str_, max_length) for str_ in list_of_strs_0])
end_time = time.time()

In [10]:
start_time = time.time()
sparse_strs_1 = sparse.vstack([onehot_string(str_, max_length) for str_ in list_of_strs_1])
end_time = time.time()

In [11]:
TPB = 16
@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [12]:
A = np.array([[random.randrange(0, 10) for i in range (2)] for j in range(2)])

In [13]:
B = np.array([[random.randrange(0, 10) for i in range (2)] for j in range(2)])

In [14]:
C = np.array([[0 for i in range (2)] for j in range(2)])

In [15]:
matmul(A, B, C)

NameError: name 'matmul' is not defined

In [19]:
num_of_chars = 208

In [20]:
a = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(40000)])
b = np.array([[random.randrange(0, 2) for i in range(num_of_chars)] for i in range(30000)])

In [21]:
b_t = np.transpose(b)

In [460]:
start_time = time.time()
c_true = np.matmul(a, b_t)
end_time = time.time()

In [466]:
end_time-start_time

465.81991839408875

In [459]:
c_true

array([[52, 52, 50, ..., 58, 52, 59],
       [39, 32, 41, ..., 46, 44, 37],
       [47, 42, 54, ..., 61, 56, 55],
       ...,
       [41, 37, 45, ..., 48, 51, 54],
       [51, 46, 54, ..., 57, 57, 53],
       [48, 42, 52, ..., 48, 52, 57]])

In [452]:
@cuda.jit
def matmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
            
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

In [462]:
blocks = 10

In [463]:
threads_x = a.shape[0]//blocks
threads_y = b.shape[0]//blocks

In [464]:
threads_x, threads_y

(4000, 3000)

In [468]:
c = np.zeros(shape=(a.shape[0], b.shape[0]))
start_time = time.time()
matmul[[1, 1], [threads_x, threads_y]](a, b_t, c)
end_time = time.time()

MemoryError: 

In [457]:
c

array([[56., 50., 57., ..., 49., 60., 58.],
       [51., 49., 55., ..., 45., 48., 57.],
       [56., 52., 57., ..., 48., 57., 55.],
       ...,
       [59., 55., 55., ..., 46., 59., 64.],
       [56., 55., 52., ..., 44., 47., 53.],
       [45., 45., 54., ..., 43., 39., 52.]])

In [150]:
def test_manual(a, b, i):
    tmp = 0
    for j in range(b.shape[0]):
        tmp+=abs(a[i]-b[j])
        
    return tmp